In [1]:
from gensim.models import KeyedVectors
import csv
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import igraph
from gensim.test.utils import datapath

### Get The embeddings

In [2]:
wv_from_text_PMC = KeyedVectors.load_word2vec_format("../../../../../data/PMC/wikipedia-pubmed-and-PMC-w2v.bin", binary=True)
wv_from_text_GV  = KeyedVectors.load_word2vec_format("../../../../../data/Glove_Gigaword/model.txt", binary=False)
wv_from_text_GM  = KeyedVectors.load_word2vec_format("../../../../../data/Gensim_wiki/model.txt", binary=False)
wv_from_text_FT  = KeyedVectors.load_word2vec_format("../../../../../data/FastText_wiki/model.txt", binary=False)

FileNotFoundError: [Errno 2] No such file or directory: '../../../../../data/PMC/wikipedia-pubmed-and-PMC-w2v.bin'

In [4]:
nb_top_terms=20

### Cosine for n-top terms with the frequency

In [15]:
def cosineSimilarity(f,model):
    cr = csv.reader(open(f,"r"))
    topterms=[]
    for row in cr:
        topterms=row
        break
    print(topterms)
    newtopterms=['']*nb_top_terms
    i=0
    for w in topterms:
        if (w not in model.vocab):
             for v in model.vocab:
                if ((w in v)and (w[0]==v[0]) and (w[1]==v[1])):
                    newtopterms[i]=v
                    #print(newtopterms[i])
                    break

        else:
            newtopterms[i]=w
            #print(newtopterms[i])

        i=i+1

    #print(newtopterms)
    terms=[]
    for w in newtopterms:
        if not(w==''):
            terms.append(w)
    
    print(len(terms))
    print(terms)
    rows = []

    for w in terms:
        for ww in terms:
            if(w == ww): rows.append(dict(w1=w, w2=ww, d= 0))
            else :
                rows.append(dict(w1=w, w2=ww, d=model.similarity(w, ww)))
    #Compute the cosin similarity matrix 
    CosineMatrix = pd.DataFrame(rows).pivot("w1", "w2", "d")
    return CosineMatrix

In [16]:
f="../../../../../data/PubMed5/Frequency/top_terms_amd.csv"
cosine_Glove=cosineSimilarity(f,wv_from_text_GV)
cosine_Gensim=cosineSimilarity(f,wv_from_text_GM)
cosine_FT=cosineSimilarity(f,wv_from_text_FT)
cosine_PMC=cosineSimilarity(f,wv_from_text_PMC)

['macular', 'ey', 'visual', 'retin', 'acuiti', 'month', 'improv', 'diabet', 'neovascular', 'degen', 'signif', 'edema', 'intravitr', 'thick', 'choroid', 'inject', 'measur', 'laser', 'chang', 'outcom']
16
['macular', 'ey', 'visual', 'retina', 'month', 'improv', 'diabetes', 'degenerate', 'significant', 'edema', 'thick', 'inject', 'measure', 'laser', 'chang', 'outcome']
['macular', 'ey', 'visual', 'retin', 'acuiti', 'month', 'improv', 'diabet', 'neovascular', 'degen', 'signif', 'edema', 'intravitr', 'thick', 'choroid', 'inject', 'measur', 'laser', 'chang', 'outcom']
18
['macular', 'ey', 'visual', 'retina', 'month', 'improv', 'diabetes', 'neovascularization', 'degenerate', 'significant', 'edema', 'thick', 'choroid', 'inject', 'measure', 'laser', 'chang', 'outcome']
['macular', 'ey', 'visual', 'retin', 'acuiti', 'month', 'improv', 'diabet', 'neovascular', 'degen', 'signif', 'edema', 'intravitr', 'thick', 'choroid', 'inject', 'measur', 'laser', 'chang', 'outcom']
17
['macular', 'ey', 'visual'

In [17]:

def computeCosine2Vec(v,w):
    a = np.array(v)
    b = np.array(w)
    aa = a.reshape(1,len(v))
    ba = b.reshape(1,len(v))
    cosim = cosine_similarity(aa, ba)
    return cosim[0][0]

In [18]:

def computeCosineSimilarities(f,model1,model2,model3,model4):
    cr = csv.reader(open(f,"r"))
    for row in cr:
        topterms=row
        break
        
    newtopterms=['']*len(topterms)
    i=0
    for w in topterms:
        if(w in model1.vocab)and (w in model2.vocab) and (w in model3.vocab)and (w in model4.vocab):
            newtopterms[i]=w
            i=i+1
        if(w not in model1.vocab):
            for v in model1.vocab:
                if (w in v)and(w[0]==v[0]):
                    if ((v in model2.vocab) and (v in model3.vocab)and (v in model4.vocab)):
                        newtopterms[i]=v
                        i=i+1
                        break

    #probleme avec significantli moi je dis on le vire :3
    #Compute the cosin similarity matrix     
    rows = []
    terms=[]
    for w in newtopterms:
        if not(w==''):
            terms.append(w)
    
    print(len(terms))
    print(terms)
    
    for w in terms:
        vect_w=list(model1[w])+list(model2[w])+list(model3[w])+list(model4[w])
        for ww in terms:
            if(w == ww): rows.append(dict(w1=w, w2=ww, d= 0))
            else :
                vect_ww= list(model1[ww]) + list(model2[ww]) + list(model3[ww]) + list(model4[ww])
                cos=computeCosine2Vec(vect_w,vect_ww)
                rows.append(dict(w1=w, w2=ww, d=cos))
     
    cosindf = pd.DataFrame(rows).pivot("w1", "w2", "d")
    return cosindf

In [19]:
cosine_GV_GM_FT_PMC=computeCosineSimilarities(f,wv_from_text_GV,wv_from_text_GM,wv_from_text_FT,wv_from_text_PMC)

16
['macular', 'ey', 'visual', 'retina', 'month', 'improv', 'diabetes', 'degenerate', 'significant', 'edema', 'thick', 'inject', 'measure', 'laser', 'chang', 'outcome']


In [20]:
#il faudra crée un fichier avec la frequence des tops termes pour les noeuds 
def graphCos(cosindf,pathfile, color):
    #Topterms 
    vect=cosindf.index
    topterms=['']*len(vect)
    for i in range(len(vect)):
        topterms[i]=vect[i]
    
    #frequences
    frequences=[0]*len(topterms)
    i=0
    with open(pathfile) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        for row in csvReader:
            if not(len(row)==0):
                #print(row[0])
                for v in topterms :
                    if(row[0] in v)and(row[0][0]==v[0]):
                        frequences[i]=float(row[1])
                        #print(frequences[i])
                        i=i+1
                        break

    # Make a graph from the adjacency matrix
    g = igraph.Graph.Weighted_Adjacency((cosindf.values > 0.3).tolist(), mode = "undirected")

    # Set words as vertices label
    g.vs["name"] = topterms
    g.vs["label"] = g.vs["name"]
    g.vs["color"] = color

    # Set size as vertices size by frequence of term in documents
    size_vertex =frequences 
    #normalize the vectors
    size_vertex_norm = [float(i)/(sum(size_vertex) - size_vertex[0])*800  for i in size_vertex]
    g.vs["size"] = size_vertex_norm

    # Set cosine similarity as edge width
    edge_w = [0]*len(g.get_edgelist())
    
    for ie, (i,j) in enumerate(g.get_edgelist()):
        if i == j:
            edge_w[ie] = 0
        else:
            edge_w[ie] = (cosindf.iloc[i][j] + 2)
    edge_w_color = [""]*len(g.get_edgelist())
    #il faut trouver un moyen de trouver les intervalles automatiquement
    for ie, edge_value in enumerate(edge_w):
        if (edge_value > 2.4):
            edge_w_color[ie] = "#126600" 
        elif (edge_value > 2.35 and edge_value < 2.4):
            edge_w_color[ie] = "#25cc00"
        else:
            edge_w_color[ie] = "#2eff00"

    g.es["weight"] = edge_w
    g.es["width"] = g.es["weight"]
    g.es["color"] = edge_w_color

    return g

In [21]:
# Variables correspondant aux couleurs des noeuds
# A modifier si besoin
colorNodeEmbeddingBasic = '#fbff28' #jaune
colorNodePMC = '#ff0000' #rouge
colorNodeGlove = '#287eff' #bleue
colorNodeFastText = '#d000f9' #violet
colorNodeGensim = '#f2b204' #orange

In [23]:
#il va falloir enregistrer le mot est sa frequence pour verifier si le mot n'a pas était viré
g = graphCos(cosine_GV_GM_FT_PMC,'../../../../../data/PubMed5/Frequency/freq_amd.csv', colorNodeEmbeddingBasic)
# plot the graph
igraph.summary(g)
out = igraph.plot(g, margin= 50, bbox = (700,700))
out.save('../../../../../output_Graph/deuxiemeversion/Graph_All_embeddings_AMD.png')

IGRAPH UNW- 16 13 -- 
+ attr: color (v), label (v), name (v), size (v), color (e), weight (e), width (e)


In [24]:
#il va falloir enregistrer le mot est sa frequence pour verifier si le mot n'a pas était viré
g = graphCos(cosine_GV_GM_FT_PMC,'../../../../../data/PubMed5/Frequency/freq_amd.csv', colorNodeEmbeddingBasic)
# plot the graph
igraph.summary(g)
out = igraph.plot(g, margin= 50, bbox = (700,700))
out.save('../../../../../output_Graph/deuxiemeversion/Graph_All_embeddings_AMD.png')

IGRAPH UNW- 16 13 -- 
+ attr: color (v), label (v), name (v), size (v), color (e), weight (e), width (e)


In [25]:
#il va falloir enregistrer le mot est sa frequence pour verifier si le mot n'a pas était viré
g = graphCos(cosine_Glove,'../../../../../data/PubMed5/Frequency/freq_amd.csv', colorNodeGlove)
# plot the graph
igraph.summary(g)
out = igraph.plot(g, margin= 50, bbox = (700,700))
out.save('../../../../../output_Graph/deuxiemeversion/Graph_GV_embeddings_AMD.png')

IGRAPH UNW- 16 13 -- 
+ attr: color (v), label (v), name (v), size (v), color (e), weight (e), width (e)


In [26]:
#il va falloir enregistrer le mot est sa frequence pour verifier si le mot n'a pas était viré
g = graphCos(cosine_Gensim,'../../../../../data/PubMed5/Frequency/freq_amd.csv', colorNodeGensim)
# plot the graph
igraph.summary(g)
out = igraph.plot(g, margin= 50, bbox = (700,700))
out.save('../../../../../output_Graph/deuxiemeversion/Graph_GM_embeddings_AMD.png')

IGRAPH UNW- 18 35 -- 
+ attr: color (v), label (v), name (v), size (v), color (e), weight (e), width (e)


In [27]:
#il va falloir enregistrer le mot est sa frequence pour verifier si le mot n'a pas était viré
g = graphCos(cosine_FT,'../../../../../data/PubMed5/Frequency/freq_amd.csv', colorNodeFastText)
# plot the graph
igraph.summary(g)
out = igraph.plot(g, margin= 50, bbox = (700,700))
out.save('../../../../../output_Graph/deuxiemeversion/Graph_FT_embeddings_AMD.png')

IGRAPH UNW- 17 22 -- 
+ attr: color (v), label (v), name (v), size (v), color (e), weight (e), width (e)


In [28]:
#il va falloir enregistrer le mot est sa frequence pour verifier si le mot n'a pas était viré
g = graphCos(cosine_PMC,'../../../../../data/PubMed5/Frequency/freq_amd.csv', colorNodePMC)
# plot the graph
igraph.summary(g)
out = igraph.plot(g, margin= 50, bbox = (700,700))
out.save('../../../../../output_Graph/deuxiemeversion/Graph_PMC_embeddings_AMD.png')

IGRAPH UNW- 20 18 -- 
+ attr: color (v), label (v), name (v), size (v), color (e), weight (e), width (e)


In [29]:
fichiersimilarite='wordsim353.tsv'
index=["wordsim353"]

similaritie_GV = wv_from_text_GV.evaluate_word_pairs(datapath(fichiersimilarite))
similaritie_GM = wv_from_text_GM.evaluate_word_pairs(datapath(fichiersimilarite))
similaritie_FT = wv_from_text_FT.evaluate_word_pairs(datapath(fichiersimilarite))
similaritie_PM = wv_from_text_PMC.evaluate_word_pairs(datapath(fichiersimilarite))
    
d =  {'Glove':similaritie_GV[1][0] ,'Gensim': similaritie_GM[1][0] , 'FastText': similaritie_FT[1][0] , 'PMC' :similaritie_PM[1][0]}
df_wordsim = pd.DataFrame(data=d,index=index)
df_wordsim

,Glove,Gensim,FastText,PMC
wordsim353,0.555504,0.673403,0.672899,0.529629


In [33]:
f="../../../../../data/eval_files/wordsim_relatedness_goldstandard.txt"
similaritie_GV = wv_from_text_GV.evaluate_word_pairs(f)
similaritie_GM = wv_from_text_GM.evaluate_word_pairs(f)
similaritie_FT = wv_from_text_FT.evaluate_word_pairs(f)
similaritie_PM = wv_from_text_PMC.evaluate_word_pairs(f)

d =  {'Glove': similaritie_GV[1][0] ,'Gensim': similaritie_GM[1][0] , 'FastText': similaritie_FT[1][0] , 'PMC' :similaritie_PM[1][0]}
df_wordsim_similarity_goldstandard = pd.DataFrame(data=d,index=["wordsim_relatedness_goldstandard"])
df_wordsim_similarity_goldstandard

,Glove,Gensim,FastText,PMC
wordsim_relatedness_goldstandard,0.535715,0.61602,0.634231,0.540422


In [31]:
f="../../../../../data/eval_files/wordsim_similarity_goldstandard.txt"
similaritie_GV = wv_from_text_GV.evaluate_word_pairs(f)
similaritie_GM = wv_from_text_GM.evaluate_word_pairs(f)
similaritie_FT = wv_from_text_FT.evaluate_word_pairs(f)
similaritie_PM = wv_from_text_PMC.evaluate_word_pairs(f)

d =  {'Glove': similaritie_GV[1][0] ,'Gensim': similaritie_GM[1][0] , 'FastText': similaritie_FT[1][0] , 'PMC' :similaritie_PM[1][0]}
df_wordsim_similarity_goldstandard = pd.DataFrame(data=d,index=["wordsim_similarity_goldstandard"])
df_wordsim_similarity_goldstandard

,Glove,Gensim,FastText,PMC
wordsim_similarity_goldstandard,0.589795,0.708539,0.696419,0.540508


In [34]:
def csv_to_eval_file1(f_input,f_output):
    df_eval=pd.read_csv(f_input)
    del df_eval['CUI1']
    del df_eval['CUI2']
    df_eval = df_eval[['TERM1', 'TERM2', 'Mean']]
    df_eval.to_csv(f_output,sep='\t',index=False)

In [35]:
def csv_to_eval_file2(f_input,f_output):
    df_eval=pd.read_csv(f_input)
    del df_eval['CUI1']
    del df_eval['CUI2']
    del df_eval ['Stdev']
    df_eval = df_eval[['Term1', 'Term2', 'Mean']]
    df_eval.to_csv(f_output,sep='\t',index=False)

In [36]:
f_input_MSRS="../../../../../data/eval_files_BioMed/Input/MayoSRS.csv"
f_output_MSRS="../../../../../data/eval_files_BioMed/Output/MayoSRS.csv"
csv_to_eval_file1(f_input_MSRS,f_output_MSRS)
similaritie_PM_MSRS = wv_from_text_PMC.evaluate_word_pairs(f_output_MSRS)

In [38]:
f_input_UMNSRS_rel="../../../../../data/eval_files_BioMed/Input/UMNSRS_relatedness.csv"
f_output_UMNSRS_rel="../../../../../data/eval_files_BioMed/Output/UMNSRS_relatedness.csv"
#csv_to_eval_file2(f_input_UMNSRS_rel,f_output_UMNSRS_rel)
df_eval=pd.read_csv(f_input_UMNSRS_rel)
del df_eval['CUI1']
del df_eval['CUI2']
del df_eval ['Stdev']
df_eval = df_eval[['Term1', 'Term2', 'Mean']]
df_eval.to_csv("../../../../../data/eval_files_BioMed/Output/UMNSRS_relatedness.csv",sep='\t',index=False)
similaritie_PM_UMNSRS_rel = wv_from_text_PMC.evaluate_word_pairs("../../../../../data/eval_files_BioMed/Output/UMNSRS_relatedness.csv")

In [39]:
f_input_UMNSRS_sim="../../../../../data/eval_files_BioMed/Input/UMNSRS_similarity.csv"
f_output_UMNSRS_sim="../../../../../data/eval_files_BioMed/Output/UMNSRS_similarity.csv"
#csv_to_eval_file2(f_input_UMNSRS_sim,f_output_UMNSRS_sim)
df_eval=pd.read_csv("../../../../../data/eval_files_BioMed/Input/UMNSRS_similarity.csv")
del df_eval['CUI1']
del df_eval['CUI2']
del df_eval ['Stdev']
df_eval = df_eval[['Term1', 'Term2', 'Mean']]
df_eval.to_csv("../../../../../data/eval_files_BioMed/Output/UMNSRS_similarity.csv",sep='\t',index=False)
similaritie_PM_UMNSRS_sim = wv_from_text_PMC.evaluate_word_pairs("../../../../../data/eval_files_BioMed/Output/UMNSRS_similarity.csv")


In [40]:
d =  {'MayoSRS': similaritie_PM_MSRS[1][0] ,'UMNSRS_relatedness': similaritie_PM_UMNSRS_rel[1][0] , 'UMNSRS_similarity': similaritie_PM_UMNSRS_sim[1][0]}
df_PMC = pd.DataFrame(data=d,index=["PubMed Central"])

In [41]:
df_PMC

,MayoSRS,UMNSRS_relatedness,UMNSRS_similarity
PubMed Central,0.395553,0.501167,0.560862
